The fear of the LORD is the beginning of knowledge, but fools despise wisdom and discipline. Proverbs 1:7

-------

# Welcome to "AI for All"

Lecture Notes by idebtor@gmail.com, Handong Global University


# Chapter 5-5 로지스틱 회귀 뉴론 만들기

    5.1 퍼셉트론
    5.2 시그모이드 함수
    5.3 로지스틱 손실함수
    5.4 이진 분류를 위한 데이터셋 준비
    5.5 로지스틱 회귀 뉴론 만들기
    5.6 로지스틱 회귀 뉴론으로 단일층 신경망 만들기
    5.7 사이킷런의 로지스틱 회귀

당뇨병 환자 데이터셋을 이용하여 선형 회귀를 할 때는 데이터셋 전체를 사용하여 모델을 훈련했습니다. 그런데, 여기서 잠시 멈추어 생각을 해보아야 합니다. 훈련된 모델이 실전에서 얼마나 좋은 성능을 내는지 어떻게 알 수 있을까요? 여러분이 만든 모델의 성능을 평가하지 않고, 실전에 투입하면 잘못된 결과를 초래할 수도 있으니 위험합니다. 

또 훈련 데이터셋으로 학습된 모델을 다시 훈련 데이터셋로 평가하면 어떨까요? 만약 모델이 훈련 데이터셋를 몽땅 외워버렸다면(실제 이런 것이 가능합니다) 평가를 해도 의미가 없을 것입니다. 

그러면 어떻게 해야 모델의 성능을 제대로 평가할 수 있을까요? 모델을 만들기 전에 성능 평가에 대해 알아보도록 합니다. 

## 1. 모델 성능 평가를 위한 훈련 셋과 테스트 셋

훈련된 모델의 실전 성능을 일반화 성능(generalization performance)이라고 부릅니다. 만약, 훈련한 데이터셋으로 성능을 평가하면 당연히 좋은 성능이 나올 것입니다. 이런 성능 평가를 '과도하게 낙관적으로 일반화 선응을 추정한다'고 말합니다. 

그러면, 올바르게 모델의 성능 축정하려면 어떻게 해야 할까요? 훈련 데이터 셋을 두 부분으로 나누어 하는 훈련에, 또 다른 하나는 테스트에 사용하면 됩니다. 이때 각각의 데이터셋으로 구분하여 하나는 훈련셋(training-set) 다른 하나는 테스트셋(test set)이라고 부릅니다. 

훈련 데이터셋을 훈련셋과 테스트셋으로 나눌 때는 다음 2가지 규칙을 지켜야 합니다. 

> 훈련 데이터셋을 훈련셋과 테스트셋으로 나누는 규칙
> - 훈련 데이터셋을 나눌 때는 테스트 셋보다 훈련셋이 더 많아야 한다. 
> - 훈련 데이터셋을 나누기 전에 양성, 음성 클래스가 훈련셋과 테스트셋의 어느 한쪽에 몰리지 않도록 골고루 섞어야 한다. 

위 과정은 사이킷런에 준비되어 있는 도구를 사용하면 편리하게 진행할 수 있습니다. 그러면, 지금부터 훈련 데이터셋을 둘로 분리하겠습니다. 

## 2 훈련셋과 테스트셋으로 나누기

훈련 데이터셋을 훈련셋과 테스트셋으로 나눌 때는 양성, 음성 클래스가 두 셋들에 고르게 분포하도록 만들어야 합니다. 예를 들면, cacner 데이터셋를 보면 음성 클래스(정상 종양)와 양성클래스(악성 종양)의 샘플 갯수각 각각 213, 357 개입니다. 이 클래스 비율이 두 셋에도 그대로 유지되어야 올바르게 학습할 수 있고, 또한 성능도 제대로 평가할 것입니다. 

지금부터 음성 클래스와 양성 클래스의 비율을 일정하게 유지하면서 훈련 데이터셋을 둘 로 나누어 보겠습니다. 

#### 1. train_test_split() 함수로 훈련 데이터셋 나누기

먼저 `sklearn.model_selection`모듈에서 `train_test_split()`함수를 `import`합니다.  `train_test_split()`함수는 기본적으로 입력된 훈련 데이터 셋을 훈련셋 75%, 테스트셋 25%의 비율로 나누어 줍니다. 

In [5]:
import numpy as np
from sklearn.model_selection import train_test_split

그런 다음 train_test_split() 함수에 입력 데이터셋 x, 타깃 데이터셋 y와 그 밖의 설정을 매개변수로 지정하면 됩니다. 

In [7]:
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()
x = cancer.data
y = cancer.target

x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.2, random_state=42)

매개 변수 설정에 대한 내용은 다음과 같습니다. 

- stratify=y : stratify는 훈련 데이터셋을 나눌 때 클래스 비율을 동일하게 만듭니다. train_test_split()함수는 기본적으로 데이터를 나누기 전에 섞지만 일부 클래스 비율이 불균형한 경우에는 stratify를 y로 지정해야 합니다.

- test_size = 0.2 : train_test_split() 함수는 기본적으로 훈련 데이터셋을 3:1 비율로 나눕니다. 하지만 필요한 경우 이 비율을 조절하고 싶을 때도 있습니다. 그러한 경우 test_size 매개변수에 테스트셋의 비율을 전달하면 비율을 조절할 수 있습니다. 여기서 0.2는 테스트셋의 크기를 20%로 하겠다는 것입니다. 

- random_state=42 : train_test_split() 함수는 무작위로 데이터셋을 섞은 다음 나눕니다. 여기서는 섞은 다음 나눈 결과가 항상 일정하도록 random_state 매개변수에 난수 초깃값을 42로 지정한 것입니다. (이렇게 하는 것이 디버깅할 때 유익합니다.)



#### 2. 결과 확인하기



그러면, 훈련 데이터셋이 훈련셋과 테스트셋으로 잘 나누어졌는디, 그 비율을 확인해 보겠습니다. `shape`속성을 이용해 확인해 보면, 각각의 셋이 4:1의 비율(`test_size = 0.2`)로 잘 나누어졌습니다. 

```
(455, 30) (114, 30)
```

In [8]:
print(x_train.shape, x_test.shape)

(455, 30) (114, 30)


#### 3. unique()함수로 훈련셋의 타깃 확인하기

넘파이의 unique()함수로 훈련셋의 타깃 안에 있는 클래스의 개수를 확인해보니 전체 훈련 데이터셋의 클래스 비율과 거의 비슷한 구성입니다. (양성 클래스가 음성 클래스보다 1.7배정도 많습니다) 클래스의 비율이 그대로 유지되고 있습니다. 

```
(array([0, 1]), array([170, 285], dtype=int64))
(array([0, 1]), array([42, 72], dtype=int64))
```

In [ ]:
np.unique(None, None)

In [ ]:
np.unique(None, None)

## 3. 로지스틱 회귀 구현하기 

이제 훈련셋이 준비되었으니, 본격적으로 로지스틱 회귀를 구현하도록 합니다. 로지스틱 회귀는 순방향으로 데이터가 흘러가는 과정과 가중치를 조정하기 위해 역방향으로 데이터가 흘러가는 과정을 구현해야 합니다. 순방향 계산부터 역방향 계산까지 순서대로 구현하도록 합니다. 

여기서 만들 `LogisticNeuro`n 클래스의 매소드는 앞에서 구현한 `Neuron` 클래스의 `__init__(), forpass(), backprop()` 메소드와 거의 동일합니다. 다음을 입력하십시오. 

In [ ]:
class LogisticNeuron:
    def __init__(self):
        self.w = None
        self.b = None
    
    def forpass(self, x):
        z = np.sum(x * self.w) + self.b  #np.dot(s, self.w)
        return z
    
    def backprop(self, x, err):
        w_grad = x * err        # 가중치에 대한 gradient를 계산함
        b_grad = 1 * err        # 편향에 대한 gradient를 계산함
        return w_grad, b_grad 

이 코드는 앞에서 코딩한 `Neuron`클래스와 별로 다르지 않지만, 몇가지 세부 설정이 바뀌었습니다. 다음은 각 메소드가 `Neuron`클래스와 비교하여 어떻게 달라졌는지 설명합니다. 

- `__init__()` __메소드는 가중치와 편향을 미리 초기화하지 않습니다.__ 여기서 `__int__()`메소드를 보면, 입력 데이터의 특성이 많아 가중치를 미리 초기화 하지 않았습니다. 가중치는 나중에 입력 데이터를 보고, 특성 개수에 맞게 결정합니다. 

- `forpass()` __ 메소드를 보면 가중치의 입력 특성의 곱을 모두 더하기 위해 np.sum()을 사용했습니다. `x * self.w` 에서 `x`와 `w`는 1차원 넘파이 배열인데, 넘파이 배열에서 사칙 연산을 적용하면 자동으로 배열의 요소끼리 계산합니다. 예를 들면, 넘파이 배열 `[1, 2, 3]` 과 `[4, 5, 6]`을 더하면, `[4, 6, 8]`이라는 넘파이 배열이 나오고 곱하면 `[3, 8, 15]`배열이 나옵니다. 또 넘파이 배열을 sum()함수의 인자로 전달하면, 각 요소를 모두 더한 값을 반환합니다. 

In [ ]:
a = np.array([1, 2, 3])
b = np.array([3, 4, 5])
print(a + b)
print(a * b)
print(np.sum(a * b))

로지스틱 뉴론이 구현되었습니다. 이제 훈련하고 예측하는 일만 남았습니다. 

## 4. 훈련하는 메소드 구현하기

훈련을 수행하는 fit()메소드를 구현하기로 합니다. 

#### 1. fit() 메소드 구현하기 

`fit()` 메소드의 가장 기본 구조는 `Neuron`클래스와 같습니다. 다만 활성화 함수가 추가된 점이 다릅니다. 로지스틱 회귀에는 이진 분류를 위한 활성화 함수인 시그모이드 함수가 필요했던 것을 기억하십시오. `activation()`메소드는 바로 다음에 구현합니다. 역방향 계산에는 로지스틱 손실 함수의 도함수를 적용합니다. 앞에서 초기화하디 않은 가중치는 `np.ones()` 함수를 이용하여 간단히 1로 초기화하고 편향은 0으로 초기화 합니다. 

In [ ]:
def fit(self, x, y, epochs = 100):
    self.w = np.ones(x.shape[1])        # 가중치 초기화 
    self.b = 0                          # 편향 초기화
    for i in range(epochs):             # epochs만큰 반복하기
        for x_i, y_i in zip(x, y):      # 모든 샘플에 대해 반복하기
            z = self.forpass(x_i)       # 순방향 계산
            a = self.activation(z)      # 활성화 함수 적용
            err = -(y_i - a)            # 오차 계산
            w_grad, b_grad = self.backprop(x_i, err)  # 역방향 계산
            self.w -= w_grad            # 가중치 조정
            self.b -= b_grad            # 편향 조정
    

#### 2. activation() 메소드 구현하기 

activation() 메소드에는 시그모이드 함수가 사용되어야 합니다. 시그모이드 함수는 자연 상수의 지수 함수를 계산하는 넘파이의 np.exp()함수를 사용하면 간단히 만들 수 있습니다. 
    

In [9]:
def activation(self, z):
    return 1 / (1 + np.exp(-z))

## 5. 예측하는 메소드 구현하기

선형 회귀에서 새로운 샘플에 대한 예측값을 계산할 때 `forpass()` 메소드를 사용했습니다. 여러 개의 샘플을 한꺼번에 예측하려면 `forpass()`메소드를 여러 번 호출하게 되는데 이 작업은 번거롭습니다. 분류에서는 활성화 함수와 임계함수도 적용해야 하므로 새로운 샘플에 대한 예측값을 계산해주는 메소드인 `predict()` 메소드를 만들어 보겠습니다. 

#### predict() 메소드 구현하기

`predict()` 메소드의 매개변수 값으로 입력값 `x`가 2차원 배열로 전달된다고 가정하고 구현합니다. 예측값은 입력값을 선형 함수, 활성화 함수, 임계 함수 순서로 통과시키면 구할 수 있습니다. 앞에서 `forpass()`와 `activation()`메소드를 이미 구현했으니 `predict()` 메소드는 다음과 같이 간단하게 구현할 수 있습니다. 

In [ ]:
def predict(self, x):
    z = [self.forpass(x_i) for x_i in x]       # 선형 함수 적용 for x
    a = self.activation(np.array(z))            # 활성화 함수 적용 for z 
    return a > 0.5   # 계단 함수 적용 a > 0.5

여기서는 z의 계산으로 파이썬의 리스트 내포(list comprehension)문법을 사용했습니다. 리스트 내포란 대괄호(`[]`)안에 `for` 문을 삽입하여 새 리스트를 만드는 간결한 문법입니다. 

`x`의 행을 하나씩 꺼내어 `forpass()` 메소드에 적용하고 그 결과를 이용하여 새 리스트 `z`로 만드는 것입니다. `z`는 곧바로 넘파이 배열로 바꾸어 `activation()` 메소드에 전달합니다. 자, 이제 로지스틱 회귀가 구현되었습니다. 

## 6. 완성된 LogisticNeuron 클래스 한 눈에 보기 

In [10]:
class LogisticNeuron:
    def __init__(self):
        self.w = None
        self.b = None
    
    def forpass(self, x):
        z = np.sum(x * self.w) + self.b
        return z
    
    def backprop(self, x, err):
        w_grad = x * err        # 가중치에 대한 gradient를 계산함
        b_grad = 1 * err        # 편향에 대한 gradient를 계산함
        return w_grad, b_grad 

    def activation(self, z):
        z = np.clip(z, -500, 500)       # 오버플로우 방지를 위해
        return 1 / (1 + np.exp(-z)) 

    def fit(self, x, y, epochs = 100):
        self.w = np.ones(x.shape[1])        # 가중치 초기화 
        self.b = 0                          # 편향 초기화
        for i in range(epochs):             # epochs만큰 반복하기
            for x_i, y_i in zip(x, y):      # 모든 샘플에 대해 반복하기
                z = self.forpass(x_i)       # 순방향 계산
                a = self.activation(z)      # 활성화 함수 적용
                err = -(y_i - a)            # 오차 계산
                w_grad, b_grad = self.backprop(x_i, err)  # 역방향 계산
                self.w -= w_grad            # 가중치 조정
                self.b -= b_grad            # 편향 조정

    def predict(self, x):
        z = [self.forpass(x_i) for x_i in x]   # 선형 함수 적용
        a = self.activation(np.array(z))       # 활성화 함수 적용
        return a > 0.5                   

## 7. 로지스틱 회귀 모델 훈련시키기

이제 준비한 데이터셋을 사용하여 로지스틱 회귀 모델을 훈련해보고 정확도를 측정해보겠습니다. 

#### 1. 모델 훈련 하기

모델을 훈련하는 방법은 `Neuron`과 동일합니다. `LogisticNeuron` 클래스의 객체를 만든 다음, 훈련셋과 함께 `fit()`메소드를 호출하면 됩니다. 

In [19]:
neuron = LogisticNeuron()
neuron.fit(x_train, y_train)

#### 2. 테스트셋 사용해 모델의 정확도 평가하기

위 코드를 통해 훈련이 끝난 모델에 테스트셋(x_test)을 사용해 예측값을 넣고 예측한 값이 레이블 y_test와 맞는지 비교하여 정확도(0.82)를 계산합니다.  

먼저, x_test 데이터 셋으로 결과를 예측하고, 출력하십시오. 

In [21]:
neuron.predict(x_test)

array([False,  True, False, False, False, False,  True, False, False,
       False,  True, False,  True, False, False, False, False,  True,
       False,  True, False, False,  True,  True,  True, False, False,
        True,  True, False,  True,  True,  True,  True, False,  True,
        True,  True, False, False,  True,  True,  True, False, False,
       False,  True,  True,  True, False,  True, False,  True, False,
        True,  True,  True, False, False,  True,  True,  True,  True,
        True, False, False, False,  True, False,  True,  True,  True,
        True, False, False, False,  True,  True,  True, False, False,
        True, False,  True, False,  True,  True,  True, False, False,
        True,  True, False, False, False,  True, False,  True, False,
       False, False, False,  True,  True, False, False,  True, False,
        True, False,  True, False,  True,  True])

이제 위의 결과를 레이블 y_test와 비교하여 정학도를 계산해야 합니다. 우선, y_test를 출력해보십시오. 

In [22]:
y_test


array([0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 1])

x_test로 예측한 결과와 y_test를 비교하여 얼마나 맞았는지 비교하여, 그 결과로 이 모델의 정확도를 계산하십시오. 

In [18]:
neuron.predict(x_test) == y_test

array([ True,  True,  True, False,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False, False,  True,
       False,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True, False,  True,  True,  True,  True,  True, False,
        True,  True, False,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True, False,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True, False, False,
        True,  True,  True,  True,  True,  True,  True, False, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [23]:
np.sum(neuron.predict(x_test) == y_test) / len(x_test)

0.8245614035087719

혹은 더 간단히...

In [24]:
np.mean(neuron.predict(x_test) == y_test)

0.8245614035087719

`predict()` 메소드의 반환값은 `True`나 `False`로 채원(m,)크기의 배열이고, `y_test()`는 0 또는 1로 채워진 (m,)크기의 배열이므로 바로 비교할 수 있습니다. `np.mean()`함수는 매개변수 값으로 전달한 비교문 결과(넘파이 배열)의 평균을 계산합니다. 즉, 계산 결과 0.82... 는 올바르게 예측한 샘플의 비율이 됩니다. 이를 정확도(accuracy)라고 합니다. 

로지스틱 회귀가 제대로 구현된 것 같습니다. 아주 훌륭하진 않지만 82%의 정확도가 나왔네요. 사실 우리가 구현한 `LogisticNeuron`클래스의 성능은 좋은 편이 아닙니다. 실전에서는 사이킷런과 같은 안정적인 패키지 사용을 권합니다. 지금은 학습을 위해 구현한 것이므로 성능에 크게 신경쓰지 않아도 됩니다. 

이제 마지막으로 하나의 층(layer)을 가진 신경망을 구현해보겠습니다. 

## 참고문헌

1. 케라스 창시자에게 배우는 딥러닝, 프랑소와 숄레, 길벗
1. 핸즈온 머신러닝, 오렐리앙 제롱, 한빛미디어
1. 딥러닝 입문, 박해선, 이지스 퍼블리싱
1. 파이썬으로 배우는 기계학습, 김영섭, K-MOOC

-------
__Be joyful always!__ 1 Thes.5:16